In [1]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd
import fitz
import os
import boto3
import io
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

F:\anaconda3\envs\python_3_7\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



# Code to Convert PDF to PNG

In [2]:
def convert_pdf2img(input_file: str,pages=None):
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if str(pages) != str(None):
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        path = 'Makro Invoices/Makro Invoices/New/'
        output_file = f"{path+os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        pix.writePNG(output_file)
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

In [3]:
convert_pdf2img('Makro Invoices/Makro Invoices/Convert/4000108244 meneghello stock almacen 01.07.2022.pdf')

Deprecation: 'preRotate' removed from class 'Matrix' after v1.19 - use 'prerotate'.
Deprecation: 'getPixmap' removed from class 'Page' after v1.19 - use 'get_pixmap'.
Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


## Summary ########################################################
File:Makro Invoices/Makro Invoices/Convert/4000108244 meneghello stock almacen 01.07.2022.pdf
Pages:None
Output File(s):['Makro Invoices/Makro Invoices/New/4000108244 meneghello stock almacen 01.07.2022_page1.png']
###################################################################


['Makro Invoices/Makro Invoices/New/4000108244 meneghello stock almacen 01.07.2022_page1.png']

In [3]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [5]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [153]:
Ima = "Makro Invoices/Makro Invoices/New/10.02.20_Cialgleztrujillo E Hijos SL_302,1_page1.png"

In [154]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
       "Document_Type":"",
      "Invoice_Number":''}
img  = cv2.imread(Ima, cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
date = []
tot = []
doc_type = []
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):


        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
            print(b)
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            dat["Vat Number"] = b[0]
        elif (re.search("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
        
        if "ALBARAN" in a or "Albaran" in a:
            dat["Document_Type"] = "Albaran"
            doc_type.append("Albaran")
        elif "Factura" in a or "FACTURA" in a  or "factura" in a :
            dat["Document_Type"] = "Factura"
            doc_type.append("Factura")
        if len(doc_type)>1:
            dat["Document_Type"] = doc_type[0]
        
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{1}/\d{2}/\d{2}",a)):
            b = re.findall("\d{1}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        
        elif (re.search("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
    
        if len(date)>1:
            dat["Date"] = date[0]

        #invoice number section
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]
    
    
    
    
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a  or "TOTAL" in a or  "Total factura" in a or  "Total Factura" in a or "TOTAL FACTURA" in a or "Total CON IGIC" in a  or "IMPORTE FACTURA EU" in a or "IMPORTE FACTURA" in a or "EUROS" in a or "Importe:" in a):
            print("Case Total")
            b = a.split(" ")
            print(b)
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
            elif b[-1]=="€":
                dat["Total"] = b[-2]
                
            elif b[-1]=='€':
                dat["Total"] = b[-2]
            elif b[-1] == "|":
                dat["Total"] = b[-2]
            elif b[-1] == "EUR":
                dat["Total"] = b[-2]
            else:
                dat["Total"] = b[-1]
                print(b[-1])


                
       
                
if dat["Total"] == 0 or dat["Total"]== "©" or (dat['Total'])==1 or dat["Total"]=="SUB TOTAL"  or dat["Total"] == 'factura' or dat["Total"]=="FACTURA" or dat["Total"]== "EUROS" or dat["Total"] == "©" or dat["Total"]=="TOTAL":
        tot = []
        im = Image.open(Ima) 
        # Size of the image in pixels (size of orginal image) 
        # (This is not mandatory) 
        width, height = im.size 
        print(im.size)
        # Setting the points for cropped image 
        left = 605
        top = 600
        right = 1675
        bottom = 1590
        # Cropped image of above dimension 
        # (It will not change orginal image) 
        im1 = im.crop((left, top, right, bottom)) 
         # Shows the image in image viewer 
        im1.save("Makro Invoices/Makro Invoices/New/pp2.jpg")
        img  = cv2.imread('Makro Invoices/Makro Invoices/New/pp2.jpg', cv2.COLOR_RGB2GRAY)
        tex=(pytesseract.image_to_string(img))
        text_list = tex.split("\n")
        text_list = text_list
        print(text_list)
        for idx, a in enumerate(text_list):
            if(a.strip() != ""):
                if re.search('[0-9]{2},[0-9]{2}',a):
                    b = re.findall('[0-9]{2},[0-9]{2}',a)
                    print(b,'this is total')
                    dat['Total'] = b[0]
                
                
                
                
        
            
            
        

<class 'list'>
Case Total
['°', 'ARTIC.', '({CONTENIDO,', 'CARACTERISTICAS,', 'ET‘', 'IGIC', '|', 'TOTAL']
TOTAL
(1157, 1649)
['« 1,00', '0,50', '', '3,00', '', '3,00', '', 'Mapiestacion', '2 B76744861 ras', 'Poll -Barranco_Las Torres', '', 'fno.: 922 711 449', '', 'r transferencia): FIRMA:', '', ')S PASADAS 48 HORAS DESDE LA ENTREGA DE LA MERCANCIA', '', '']
Wall time: 7.3 s


In [155]:
dat

{'Vat Number': 'B38732012',
 'Total': 'TOTAL',
 'Date': '10/02/2020',
 'Document_Type': 'Factura',
 'Invoice_Number': ''}

In [156]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(data)

  Vat Number  Total        Date Document_Type Invoice_Number
0  B38732012  TOTAL  10/02/2020       Factura               


In [157]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data['Vat Number']= data['Vat Number'].str.replace('/', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until



,Vat Number,Total,Date,Document_Type,Invoice_Number
0,B38732012,TOTAL,10/02/2020,Factura,


In [88]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

NameError: name 'formats' is not defined

In [59]:
df2 = formats.query("last == True")

In [60]:
df2

,formato,nombre,nif,last


In [61]:
inv = 0
for i in df2['formato']:
    inv = i
if inv ==0:
    pass
else:
    factura = re.compile(inv)
    mo = factura.search(text)
    if mo is None:
        pass
    else:
        dat["Invoice_Number"]=mo.group()

In [62]:
dat

{'Vat Number': 'B38947636',
 'Total': '261,26',
 'Date': '30/05/2022',
 'Document_Type': 'Factura',
 'Invoice_Number': '0/0(006)0052/(2022)150028'}

In [237]:
im = Image.open(Ima)

In [238]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [239]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']
            
dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [240]:
if dataframes[0].shape[1] >=4 :
    this_data = pd.DataFrame(dataframes[0])

elif  dataframes[1].shape[1] >=4 :
    this_data = pd.DataFrame(dataframes[1])
  
elif dataframes[2].shape[1] >=4:
    this_data = pd.DataFrame(dataframes[2])

else:
    this_data = pd.DataFrame(dataframes[3])
    
this_data  

,,NÚM. PEDIDO,CÓDIGO ARTIC.,CÓDIGO EAN 13,"DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)",BULT.,UDS.,PRECIO X DO.,DTO.,IGIC,TOTAL
0,2,000065,TROL186,8436556395833,TROLLI SOUR OCTOPUS 120Uds.,"1,00",1,"7,800",,"3,00","7,80"
1,2,000065,BULG106,8006908003725,BULGARI BOLAS ROSAS 900gr. 100uds.,"1,00",1,"6,000",,"3,00","6,00"
2,2,000065,31011,8585004510424,MUSLI DISPLAY BARRITAS MANZANA 30gr.,"1,00",32,"0,341",,"3,00","10,91"
3,2,000065,VICE006,8033102691212,GRISBI WAFER COCO 30gr 2x1€,"1,00",24,"0,345",,"3,00","8,28"
4,2,000065,ILLY002,90399572,ILLY ISSIMO LATTE MACCHIATO DRINK 250ml.,"0,50",", 6","1,350",,"3,00","8,10"
5,2,000065,ILLY003,90399589,ILLY ISSIMO CAPPUCCINO DRINK 250ml.,"0,50",6,"1,350",,"3,00","8,10"
6,2,000065,LACA018,8410556008283,PANOPLIA CONGUITOS CHOCOLATE 20gr.,"1,00",24,"0,204",,"3,00","4,90"
7,2,000065,NEST001,5000189980891,NUTS 42gr.,"1,00",", 24","0,582",,"3,00","13,97"
8,2,000065,MARS011,400900456647,SKITTLES BOLSA CRAZY 38gr.,"1,00",14,"0,427",,"3,00","5,98"
9,2,000065,MARS021,5000159338882,SKITTLES BOLSA FRUTA 38gr.,"1,00",14,"0,427",,"3,00","5,98"


In [241]:
dat

{'Vat Number': 'B38732012',
 'Total': 'TOTAL',
 'Date': '10/02/2020',
 'Document_Type': 'Factura',
 'Invoice_Number': ''}

In [242]:
this_data.columns = this_data.columns.str.replace('SELECTED  ', '')
this_data.columns = this_data.columns.str.replace('SELECTED ', '')
this_data.columns = this_data.columns.str.replace('SELECTED', '')
this_data.columns = this_data.columns.str.replace('NOT_', '')
this_data.columns = this_data.columns.str.replace('NOT_SELECTED  ', '')
this_data.columns = this_data.columns.str.replace('NOT_SELECTED', '')
big = this_data.columns[0]
cont = this_data.columns[2]
lot = this_data.columns[1]
prec = this_data.columns[3]
cant = this_data.columns[-5]
imp = this_data.columns[-2]
importe = this_data.columns[-3]
third = this_data.columns[-4]
this_data = this_data.loc[(this_data[big] != "Número de pedido") & (this_data[big] != "*** Número de pedido") & (this_data[big]!="*** Número de") & (this_data[big]!="*** Número")]
this_data = this_data.loc[(this_data[big] != "Entregado a: TABERNA") & (this_data[big] !="Entregado a: TASCA") & (this_data[big]!="Entregado a: TABERNA OSUNA")
          & (this_data[big]!= "Entregado a: TABERNA osuna")& (this_data[big]!="Fin de número") ]
this_data = this_data.loc[(this_data[imp]!= "Spain Fecha:") & (this_data[imp]!= "Spain Fecha:0") & (this_data[imp]!= "Spain Fecha:1")]
this_data = this_data.loc[this_data[cont] !="LOT:"]
this_data = this_data.loc[this_data[cant] !="Total"]
this_data = this_data.loc[this_data[cant]!="LA LAGUNA" ]
this_data = this_data.replace('',np.nan)
this_data = this_data.dropna(axis=0, thresh=3, how="any")
limitPer = len(this_data) * .100
this_data = this_data.dropna(thresh=limitPer, axis=1)
this_data = this_data.replace(np.nan,'0')

In [243]:
json_f = {}
json_f['file'] = Ima
json_f['labels'] = dat
json_f['tables'] = this_data
print(json_f)

{'file': 'Makro Invoices/Makro Invoices/New/10.02.20_Cialgleztrujillo E Hijos SL_302,1_page1.png', 'labels': {'Vat Number': 'B38732012', 'Total': 'TOTAL', 'Date': '10/02/2020', 'Document_Type': 'Factura', 'Invoice_Number': ''}, 'tables':       NÚM. PEDIDO CÓDIGO ARTIC.  CÓDIGO EAN 13  \
0   2      000065       TROL186  8436556395833   
1   2      000065       BULG106  8006908003725   
2   2      000065         31011  8585004510424   
3   2      000065       VICE006  8033102691212   
4   2      000065       ILLY002       90399572   
5   2      000065       ILLY003       90399589   
6   2      000065       LACA018  8410556008283   
7   2      000065       NEST001  5000189980891   
8   2      000065       MARS011   400900456647   
9   2      000065       MARS021  5000159338882   
10  2      000065       CARA006  5901715000451   
11  2      000065       ORBI407  4009900512312   
12  2      000065       HALL005  7622210286222   
13  2      000065       HALL007  8416400888131   
14  0       

In [244]:
dat

{'Vat Number': 'B38732012',
 'Total': 'TOTAL',
 'Date': '10/02/2020',
 'Document_Type': 'Factura',
 'Invoice_Number': ''}

In [245]:
this_data.head()

,,NÚM. PEDIDO,CÓDIGO ARTIC.,CÓDIGO EAN 13,"DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)",BULT.,UDS.,PRECIO X DO.,IGIC,TOTAL
0,2,000065,TROL186,8436556395833,TROLLI SOUR OCTOPUS 120Uds.,"1,00",1,"7,800","3,00","7,80"
1,2,000065,BULG106,8006908003725,BULGARI BOLAS ROSAS 900gr. 100uds.,"1,00",1,"6,000","3,00","6,00"
2,2,000065,31011,8585004510424,MUSLI DISPLAY BARRITAS MANZANA 30gr.,"1,00",32,"0,341","3,00","10,91"
3,2,000065,VICE006,8033102691212,GRISBI WAFER COCO 30gr 2x1€,"1,00",24,"0,345","3,00","8,28"
4,2,000065,ILLY002,90399572,ILLY ISSIMO LATTE MACCHIATO DRINK 250ml.,"0,50",", 6","1,350","3,00","8,10"


In [264]:
str1 = 'CÓDIGO'
str2 = ''
code = ''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        code = i
print(code)

CÓDIGO ARTIC.
CÓDIGO EAN 13
CÓDIGO EAN 13


In [265]:
codingo = this_data[code]
codingo_data  =  pd.DataFrame(codingo,columns=[code])
codingo_data

,CÓDIGO EAN 13
0,8436556395833
1,8006908003725
2,8585004510424
3,8033102691212
4,90399572
5,90399589
6,8410556008283
7,5000189980891
8,400900456647
9,5000159338882


In [266]:
name = ''
index = 0
for i in this_data.columns:
    if (i == 'DESCRIPCIÓN') or (i=="Descrip. artículo") or (i=="Artículo/ Cod.cliente")or (i == 'DESCRIPCIÓN:')  or(i =="CONCEPTO")or (i=="UDS") or (i =="Concepto") or (i=="Descripción")  or (i=='DESCRIPCION') or (i=="Description articulo") or (i=="Description") or (i == "DESCRIPCIÓN ARTICULO") or (i=="Unidades Descripción")or (i=="PRODUCTO") or (i=="Articulo") or (i=='Denominación /Referencia'):
        print(i)
        match = index
        name = i
    index+=1

In [267]:
str1 ='DESCRIPCIÓN'
str2 = "Descrip. artículo"
str3 = "Artículo/ Cod.cliente"
str4 = 'DESCRIPCIÓN:'
str5 = "CONCEPTO"
str6 = "UDS"
str7 = "Concepto"
str8 = "Descripción"
str9 = "DESCRIPCION"
str10 = "Description articulo"
str11 = "Description"
str12 = "DESCRIPCIÓN ARTICULO"
str13 = "Unidades Descripción"
str14 = "PRODUCTO"
str15 = "Articulo"
str16 = 'Denominación /Referencia'
name = ''
pro = []
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
        
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        name = i
        pro.append(i)
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print(i)
        name = i
        pro.append(i)

    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio5 = fuzz.partial_ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio6 = fuzz.partial_ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print(i)
        name = i
        pro.append(i)
    ratio7 = fuzz.partial_ratio(str7.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio8 = fuzz.partial_ratio(str8.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio9 = fuzz.partial_ratio(str9.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio10 = fuzz.partial_ratio(str10.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio11= fuzz.partial_ratio(str11.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio12 = fuzz.partial_ratio(str12.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio13 = fuzz.partial_ratio(str13.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio14 = fuzz.partial_ratio(str14.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio15 = fuzz.partial_ratio(str15.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
    ratio16 = fuzz.partial_ratio(str16.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        name = i
        pro.append(i)
        
        
if len(pro)>1:
    name = pro[0]

DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
UDS.


In [268]:
this = this_data[name]

In [269]:
dataf  =  pd.DataFrame(this,columns=[name])

In [270]:
dataf

,"DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)"
0,TROLLI SOUR OCTOPUS 120Uds.
1,BULGARI BOLAS ROSAS 900gr. 100uds.
2,MUSLI DISPLAY BARRITAS MANZANA 30gr.
3,GRISBI WAFER COCO 30gr 2x1€
4,ILLY ISSIMO LATTE MACCHIATO DRINK 250ml.
5,ILLY ISSIMO CAPPUCCINO DRINK 250ml.
6,PANOPLIA CONGUITOS CHOCOLATE 20gr.
7,NUTS 42gr.
8,SKITTLES BOLSA CRAZY 38gr.
9,SKITTLES BOLSA FRUTA 38gr.


In [271]:
prec = []
str1 = "Precio"
str2 = "Base I"
str3 = "PVP"
str4 = 'Importe'
str5 = 'Euro Uni.'
str6 = "IMP"
price =''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        prec.append(i)
        price = i
        print("Im condition 1")
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        prec.append(i)
        price =i
        print("Im condition 2")
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        prec.append(i)
        price =i
    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        prec.append(i)
        price =i
    ratio5 = fuzz.partial_ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        prec.append(i)
        price =i
    ratio6 = fuzz.partial_ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        prec.append(i)
        price =i
if len(prec)>1:
    price = prec[0]
else:
    pass

PRECIO X DO.
Im condition 1


In [272]:
price_col = this_data[price]
price_data = pd.DataFrame(price_col,columns=[price])
price_data

,PRECIO X DO.
0,"7,800"
1,"6,000"
2,"0,341"
3,"0,345"
4,"1,350"
5,"1,350"
6,"0,204"
7,"0,582"
8,"0,427"
9,"0,427"


In [273]:
str1 = "DTO"
str2 = "Descuento"
str3 = "DCTO"
str4 = "Desc"
disc = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    ratio3 = fuzz.ratio(str3.lower(),str(i).lower())
    ratio4 = fuzz.ratio(str4.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        disc = i
        print("Im condition 1")
    
    elif ratio2>=75:
        print(i)
        disc = i
        print("Im condition 2")
    
    elif ratio3>=75:
        print(i)
        disc = i
        print("Im condition 3")
    
    elif ratio4>=75:
        print(i)
        disc = i
        print("Im condition 4")
    
print(disc)

In [274]:
str1 = "Cantidad"
str2 = "Cant"
str3 = "Unid"
str4 = "Unidades"
str5 = "Uds"
str6 = "Peso"
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        cat = i
        print("Im condition 1")
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        cat =i
        print("Im condition 2")
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        cat =i
    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        cat =i
    ratio5 = fuzz.partial_ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        cat =i
    ratio6 = fuzz.partial_ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        cat =i

Im condition 3
DESCRIPCIÓN DEL ARTÍCULO (CONTENIDO, CARACTERÍSTICAS, ETC...)
Im condition 5
UDS.


In [275]:
Cantidad = this_data[cat]
Cantidad_data = pd.DataFrame(Cantidad,columns=[cat])
Cantidad_data

,UDS.
0,1
1,1
2,32
3,24
4,", 6"
5,6
6,24
7,", 24"
8,14
9,14


In [276]:
str1 = 'IMPUESTO'
imposto = ''

for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        imposto = i
print(imposto)

In [277]:
if disc =='' and imposto=='' :
    df2 = pd.concat([codingo_data,dataf,price_data,Cantidad_data],axis=1, join='inner')
    df2['Discount'] = 0
    df2['IMPUESTO']=0
elif disc !='' and imposto=='':
    discount = this_data[disc]
    discount_data = pd.DataFrame(discount,columns=[disc])
    df2 = pd.concat([codingo_data,dataf,price_data,Cantidad_data,discount_data],axis=1, join='inner')
    df2['IMPUESTO']=0
else:
    discount = this_data[disc]
    discount_data = pd.DataFrame(discount,columns=[disc])
    impota = this_data[imposto]
    impota_data = pd.DataFrame(impota,columns=[imposto])
    df2 = pd.concat([codingo_data,dataf,price_data,Cantidad_data,discount_data,impota_data],axis=1, join='inner')

In [278]:
#df = pd.DataFrame([dataf,price_data], columns=['Description','Price'])
#df2 = pd.concat([dataf,price_data,Cantidad_data,discount_data],axis=1, join='inner')


DCTO

In [279]:
#data['Vat Number']= data['Vat Number'].str.replace(',', '')
df2.columns.values[0] = "Codigo"
df2.columns.values[1] = "Description"
df2.columns.values[2] = "Precio"
df2.columns.values[3] = "Cantidad"
df2.columns.values[4] = "Discount"

df2


,Codigo,Description,Precio,Cantidad,Discount,IMPUESTO
0,8436556395833,TROLLI SOUR OCTOPUS 120Uds.,"7,800",1,0,0
1,8006908003725,BULGARI BOLAS ROSAS 900gr. 100uds.,"6,000",1,0,0
2,8585004510424,MUSLI DISPLAY BARRITAS MANZANA 30gr.,"0,341",32,0,0
3,8033102691212,GRISBI WAFER COCO 30gr 2x1€,"0,345",24,0,0
4,90399572,ILLY ISSIMO LATTE MACCHIATO DRINK 250ml.,"1,350",", 6",0,0
5,90399589,ILLY ISSIMO CAPPUCCINO DRINK 250ml.,"1,350",6,0,0
6,8410556008283,PANOPLIA CONGUITOS CHOCOLATE 20gr.,"0,204",24,0,0
7,5000189980891,NUTS 42gr.,"0,582",", 24",0,0
8,400900456647,SKITTLES BOLSA CRAZY 38gr.,"0,427",14,0,0
9,5000159338882,SKITTLES BOLSA FRUTA 38gr.,"0,427",14,0,0


In [280]:
df2["Precio"]=df2["Precio"].str.replace(',','.')
df2["Cantidad"]=df2["Cantidad"].str.replace(',','.')
df2["Precio"]=pd.to_numeric(df2["Precio"])

df2
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Codigo       20 non-null     object 
 1   Description  20 non-null     object 
 2   Precio       20 non-null     float64
 3   Cantidad     20 non-null     object 
 4   Discount     20 non-null     int64  
 5   IMPUESTO     20 non-null     int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.1+ KB


In [86]:
import mysql.connector

#establishing the connection
conn = mysql.connector.connect(
   user='root', password='admin', host='localhost', database='sakila')
cursor = conn.cursor()

# Preparing SQL query to INSERT a record into the database.
sql = """INSERT INTO Dataa(
   vat_number,total,date_,document_type,invoice_number)
   VALUES (%s, %s, %s, %s, %s)"""
val = (dat["Vat Number"],dat["Total"],dat["Date"],dat["Document_Type"],dat["Invoice_Number"])
# Executing the SQL command
cursor.execute(sql,val)
conn.commit()
conn.close()


In [97]:
mycursor = con.cursor()
mycursor.execute('select * from dataa')
print(mycursor)
for tb in mycursor:
    print(tb)
#data = pd.DataFrame(tb)

CMySQLCursor: select * from dataa
('B76744861', 'www.gmfood.es,', '03-02-2020', 'Albaran', '200302180004999ALBARAN')
('B76744861', '52,27', '05/12/2020', 'Factura', '')
('B35129584', '52,27', '05/12/2020', 'Factura', '')
('B38947636', '250,82', '27/05/2022', 'Factura', '0/0(006)0052/(2022)147042')
('B38947636', '250,82', '27/05/2022', 'Factura', '0/0(006)0052/(2022)147042')
('B38947636', '261,26', '30/05/2022', 'Factura', '0/0(006)0052/(2022)150028')


In [94]:
import mysql.connector as c
import pandas as pd
con = c.connect(host='localhost',user='root',passwd="admin",database='sakila')

In [ ]:
insert into data_extraction(id_supplier,id_company,vat_client,vat_supplier,date,invoicenumber,document_type,file,data,total)
values(1,1,'B38947636','B38947636','2022-04-09','12456','Factura','Hi_This_is_Invoice','Data','100')